In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Node
class NodeRegression():

    def __init__(
        self, 
        min_samples_split=None,
        max_depth=None,
        depth=None,
        node_type=None,
        rule=None):
      
        #Data goes to the node 
        self.Y = Y 
        self.X = X

        #Hyper parameters
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

        #Depth
        self.depth = depth

        #Mean of Y 
        self.y_mean = np.mean(Y)

        #Residuals 
        self.residuals = self.Y - self.y_mean

        #Mse of the node 
        self.mse = self.mse_cal(Y, self.y_mean)

        #Count of observations in the node 
        self.n = len(Y)

        #Left and Right nodes initialisation
        self.left = None 
        self.right = None 

        #Initialisation of splits
        self.best_feature = None 
        self.best_value = None 



    #Mean Square Error
    def mse_cal(y_act, y_hat) -> float:
        
        #Total Samples
        n = len(y_act)

        #Residuals 
        res = y_act - y_hat 

        #Residual Square 
        res_2 = r * r

        #Summation of residuals 
        r_sum = np.sum(res_2)

        #Average 
        return r_sum / n



  #Moving average

  #def gini

  #def best_split

  #def grow tree

  #print tree
